In [13]:
import numpy as np
import cs_vqe as c
import ast
import os
from tqdm import tqdm

import cs_vqe_with_LCU as c_LCU

import pickle
import datetime
from copy import deepcopy

#######
import sys

In [3]:
working_dir = os.getcwd()
data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')

with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())
    
data_csvqe_results_file = os.path.join(data_dir, 'csvqe_results.txt')
with open(data_csvqe_results_file, 'r') as input_file:
    csvqe_results = ast.literal_eval(input_file.read())

In [4]:
for key in csvqe_results.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H3_STO-3G_singlet_1+          n_qubits:  3    
Be1_STO-3G_singlet            n_qubits:  5    
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H1-He1_3-21G_singlet_1+       n_qubits:  6    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H3_3-21G_singlet_1+           n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H3-N1_STO-3G_singlet          n_qubits:  13   
Mg1_STO-3G_singlet            n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
H4-N1_STO-3G_singlet_1+       n_qubits:  14   
F2_STO-3G_singlet             n_qubits:  15   
N2_STO-3G_singlet             n_qubits:  15   
O2_STO-3G_singlet             n_qubits:  15   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G

In [8]:
mol_num=1
molecules_and_qubits = [(mol_key, hamiltonians[mol_key][1])for mol_key in csvqe_results]
sorted_by_qubit_No = sorted(molecules_and_qubits, key= lambda x: x[1])
mol_key = sorted_by_qubit_No[mol_num-1][0] # UCL supercomputer indexes from 1, hence minus one here!
mol_key

'H3_STO-3G_singlet_1+'

In [9]:
### Calc Params ##

check_reduction=False

csvqe_LCU_output={}
csvqe_standard_output={}

In [11]:
true_gs = hamiltonians[mol_key][4] # ground state energy of full Hamiltonian (in Hartree)
ham_noncon = hamiltonians[mol_key][3]  # noncontextual part of Hamiltonian, found by greedy DFS
ham = hamiltonians[mol_key][2] # full Hamiltonian
gs_noncon = hamiltonians[mol_key][5]
model = list(gs_noncon[3])
fn_form = gs_noncon[4]
ground_state_params = [list(gs_noncon[1]), list(gs_noncon[2])]
removal_order = csvqe_results[mol_key][3]

In [15]:
### LCU method!
reduced_H_LCU_list = c_LCU.get_reduced_hamiltonians_LCU(ham,
                         model,
                         fn_form,
                         ground_state_params,
                         deepcopy(removal_order), 
                         check_reduction=check_reduction)

In [23]:
### Seq Rot method (standard)
reduced_H_SeqRot_list = c.get_reduced_hamiltonians(ham, #<-- CON PART ONLY
                     model,
                     fn_form,
                     ground_state_params,
                     deepcopy(removal_order))

In [46]:
### Get SeqRot Energies
SeqRot_results={}
for ind, H_std in enumerate(reduced_H_SeqRot_list):
    if len(H_std) == 1:
        Energy = H_std['']     
    else:
        ham_red_sparse = c.hamiltonian_to_sparse(H_std)
        if ham_red_sparse.shape[0] <= 64:
            Energy = min(np.linalg.eigvalsh(ham_red_sparse.toarray()))
        else:
            eig_values, eig_vectors = eigsh(ham_red_sparse, k=1, which='SA')
            Energy = min(eig_values)
    SeqRot_results[ind] = {'E':Energy , 'H':H_std}
    
SeqRot_results['exp_conditions'] = {
    'true_gs':true_gs,
    'noncon_H':ham_noncon,
    'full_tapered_H': ham,
    'gstate_noncon': [list(gs_noncon[1]), list(gs_noncon[2])],
    'gstate_noncon_Energy': gs_noncon[0],
    'model_CSVQE': list(gs_noncon[3]),
    'fn_form_CSVQE':gs_noncon[4],
    'n_qubits':   hamiltonians[mol_key][1],
    'removal_order': removal_order,
    'mol_key': mol_key
}

In [49]:
### Get LCU Energies
LCU_results={}
for ind, H_LCU in enumerate(reduced_H_LCU_list):
    if len(H_LCU) == 1:
        Energy = H_LCU['']     
    else:
        ham_red_sparse = c.hamiltonian_to_sparse(H_LCU)
        if ham_red_sparse.shape[0] <= 64:
            Energy = min(np.linalg.eigvalsh(ham_red_sparse.toarray()))
        else:
            eig_values, eig_vectors = eigsh(ham_red_sparse, k=1, which='SA')
            Energy = min(eig_values)
    LCU_results[ind] = {'E':Energy , 'H':H_LCU}
    
LCU_results['exp_conditions'] = {
    'true_gs':true_gs,
    'noncon_H':ham_noncon,
    'full_tapered_H': ham,
    'gstate_noncon': [list(gs_noncon[1]), list(gs_noncon[2])],
    'gstate_noncon_Energy': gs_noncon[0],
    'model_CSVQE': list(gs_noncon[3]),
    'fn_form_CSVQE':gs_noncon[4],
    'n_qubits':   hamiltonians[mol_key][1],
    'removal_order': removal_order,
    'mol_key': mol_key
}

In [50]:
LCU_results

{0: {'E': -2.9032129187167244, 'H': {'': -2.9032129187167244}},
 1: {'E': -2.915710429363981,
  'H': {'I': -2.1802002532456592,
   'Z': -0.7230126654710651,
   'X': -0.1350107578751797}},
 2: {'E': -2.9160184902684563,
  'H': {'II': -1.4385286390990442,
   'ZZ': -0.019199444061130826,
   'ZX': -0.07733163434390387,
   'XI': -0.07195737217001563,
   'XX': 0.051042862665487465,
   'XZ': 0.06305338570516404,
   'YY': -0.03745984341074934,
   'IX': 0.07733164795674155,
   'IZ': 0.7416716141466151,
   'ZI': -0.7422121095321959}},
 3: {'E': -2.9160184902684536,
  'H': {'III': -1.7512307459285528,
   'ZXI': 0.023568152336180023,
   'ZZZ': 0.009599722030565413,
   'ZZX': 0.03866581717195194,
   'YYI': -0.02356815233618002,
   'XXZ': -0.027790548973821078,
   'XXX': 0.018401098184557645,
   'XII': -0.07195737217001563,
   'ZIZ': -0.009599722030565413,
   'ZIX': -0.03866581717195194,
   'IXZ': 0.027790548973821074,
   'IXX': -0.018401098184557645,
   'IZI': -0.31270210682950855,
   'XZX': -0.018